# 🏢 Crunchbase Companies Dataset API

Access Bright Data's pre-collected Crunchbase Companies dataset:
- **2.3M+ companies** with **98 fields**
- Filter by funding, employees, industry, location, and more

---

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("Set BRIGHTDATA_API_TOKEN in .env file")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")
print("Setup complete!")

API Token: 7011787d-2...3336
Setup complete!


## Initialize Client

In [2]:
from brightdata import BrightDataClient

client = BrightDataClient(token=API_TOKEN)

print("Client initialized")

Client initialized


---
## Test 1: Explore Crunchbase Fields

Explore the 98 available fields.

In [ ]:
from brightdata.datasets import CrunchbaseCompanies

print("=== Crunchbase Companies Dataset ===")
print(f"Dataset ID: {CrunchbaseCompanies.DATASET_ID}")
print(f"Total fields: {len(CrunchbaseCompanies.FIELDS)}")

# High fill rate fields (most reliable for filtering)
high_fill = CrunchbaseCompanies.get_high_fill_rate_fields(min_rate=90.0)
print(f"\nHigh fill rate fields (>90%): {len(high_fill)}")
for field in high_fill[:10]:
    info = CrunchbaseCompanies.FIELDS[field]
    print(f"  - {field}: {info['type']} ({info['fill_rate']}%)")

In [ ]:
# Show fields by type
print("\n=== Fields by Type ===")
print(f"Text fields: {len(CrunchbaseCompanies.get_fields_by_type('text'))}")
print(f"Number fields: {len(CrunchbaseCompanies.get_fields_by_type('number'))}")
print(f"Array fields: {len(CrunchbaseCompanies.get_fields_by_type('array'))}")
print(f"Object fields: {len(CrunchbaseCompanies.get_fields_by_type('object'))}")
print(f"URL fields: {len(CrunchbaseCompanies.get_fields_by_type('url'))}")

In [ ]:
# Show number fields (useful for filtering)
print("\n=== Number Fields (for numeric filtering) ===")
for field in CrunchbaseCompanies.get_fields_by_type('number')[:15]:
    info = CrunchbaseCompanies.FIELDS[field]
    print(f"  - {field}: {info['description'][:50]}... ({info['fill_rate']}%)")

---
## Test 2: Get Dataset Metadata from API

In [ ]:
print("Fetching Crunchbase metadata from API...\n")

async with client:
    metadata = await client.datasets.crunchbase_companies.get_metadata()

print(f"Dataset ID: {metadata.id}")
print(f"Total fields from API: {len(metadata.fields)}")

print("\n=== Sample Fields ===")
for i, (name, field) in enumerate(list(metadata.fields.items())[:10]):
    print(f"  {name}: {field.type} - {field.description or 'N/A'}")

---
## Test 3: Filter by Operating Status

Find active companies.

In [ ]:
# Step 1: Create filter and get snapshot_id
FILTER = {
    "name": "operating_status",
    "operator": "=",
    "value": "active"
}
LIMIT = 10

print(f"Filter: {FILTER}")
print(f"Records limit: {LIMIT}\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=FILTER,
        records_limit=LIMIT
    )

print(f"Snapshot created: {snapshot_id}")

In [ ]:
# Step 2: Download data (polls until ready)
print(f"Downloading snapshot: {snapshot_id}\n")

async with client:
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"Downloaded {len(data)} companies:")
for company in data[:5]:
    print(f"\n  Name: {company.get('name', 'N/A')}")
    print(f"  Status: {company.get('operating_status', 'N/A')}")
    print(f"  Industries: {company.get('industries', 'N/A')}")
    print(f"  Employees: {company.get('num_employees', 'N/A')}")
    print(f"  Website: {company.get('website', 'N/A')}")

# Step 1: Create filter
EMPLOYEE_FILTER = {
    "name": "num_employee_profiles",
    "operator": ">",
    "value": 100
}

print(f"Filter: Companies with >100 employee profiles")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=EMPLOYEE_FILTER,
        records_limit=5
    )

print(f"Snapshot created: {snapshot_id}")

In [ ]:
# Step 2: Download data
print(f"Downloading snapshot: {snapshot_id}\n")

async with client:
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"Downloaded {len(data)} companies:")
for company in data:
    print(f"  - {company.get('name', 'N/A')}")
    print(f"    Employee profiles: {company.get('num_employee_profiles', 'N/A')}")
    print(f"    Employees: {company.get('num_employees', 'N/A')}")

In [ ]:
# Step 1: Create filter
COUNTRY_FILTER = {
    "name": "country_code",
    "operator": "=",
    "value": "USA"
}

print("Filter: US-based companies")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=COUNTRY_FILTER,
        records_limit=5
    )

print(f"Snapshot created: {snapshot_id}")

In [ ]:
# Step 2: Download data
print(f"Downloading snapshot: {snapshot_id}\n")

async with client:
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"Downloaded {len(data)} companies:")
for company in data:
    print(f"  - {company.get('name', 'N/A')} ({company.get('country_code', 'N/A')})")
    print(f"    HQ: {company.get('address', 'N/A')[:50]}..." if company.get('address') and len(company.get('address', '')) > 50 else f"    HQ: {company.get('address', 'N/A')}")

# Step 1: Create filter
FUNDED_FILTER = {
    "operator": "and",
    "filters": [
        {"name": "operating_status", "operator": "=", "value": "active"},
        {"name": "num_investors", "operator": ">", "value": 0}
    ]
}

print("Filter: Active companies with investors")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=FUNDED_FILTER,
        records_limit=5
    )

print(f"Snapshot created: {snapshot_id}")

In [ ]:
# Step 2: Download data
print(f"Downloading snapshot: {snapshot_id}\n")

async with client:
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"Downloaded {len(data)} companies:")
for company in data:
    print(f"\n  Name: {company.get('name', 'N/A')}")
    print(f"  Status: {company.get('operating_status', 'N/A')}")
    print(f"  Investors: {company.get('num_investors', 'N/A')}")
    print(f"  Industries: {company.get('industries', 'N/A')}")
    print(f"  CB Rank: {company.get('cb_rank', 'N/A')}")

In [ ]:
# Step 1: Create filter
IPO_FILTER = {
    "name": "ipo_status",
    "operator": "=",
    "value": "public"
}

print("Filter: Public companies")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=IPO_FILTER,
        records_limit=5
    )

print(f"Snapshot created: {snapshot_id}")

In [ ]:
# Step 2: Download data
print(f"Downloading snapshot: {snapshot_id}\n")

async with client:
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"Downloaded {len(data)} companies:")
for company in data:
    print(f"  - {company.get('name', 'N/A')}")
    print(f"    IPO Status: {company.get('ipo_status', 'N/A')}")
    print(f"    Stock Symbol: {company.get('stock_symbol', 'N/A')}")

from brightdata.datasets import export_json, export_csv, export

# Export to JSON
json_file = export_json(data, "crunchbase_results.json")
print(f"Exported to: {json_file}")

# Export to CSV
csv_file = export_csv(data, "crunchbase_results.csv")
print(f"Exported to: {csv_file}")

# Or use auto-detect based on extension
# export(data, "results.json")
# export(data, "results.csv")

print(f"\nRecords: {len(data)}")

In [ ]:
# Combined filter: active companies with investors
FUNDED_FILTER = {
    "operator": "and",
    "filters": [
        {"name": "operating_status", "operator": "=", "value": "active"},
        {"name": "num_investors", "operator": ">", "value": 0}
    ]
}

print("Filter: Active companies with investors")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=FUNDED_FILTER,
        records_limit=5
    )
    print(f"Snapshot: {snapshot_id}")
    
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"\nDownloaded {len(data)} companies:")
for company in data:
    print(f"\n  Name: {company.get('name', 'N/A')}")
    print(f"  Status: {company.get('operating_status', 'N/A')}")
    print(f"  Investors: {company.get('num_investors', 'N/A')}")
    print(f"  Industries: {company.get('industries', 'N/A')}")
    print(f"  CB Rank: {company.get('cb_rank', 'N/A')}")

---
## Test 7: Filter by IPO Status

In [ ]:
# Filter: public companies
IPO_FILTER = {
    "name": "ipo_status",
    "operator": "=",
    "value": "public"
}

print("Filter: Public companies")
print(f"Records limit: 5\n")

async with client:
    snapshot_id = await client.datasets.crunchbase_companies.filter(
        filter=IPO_FILTER,
        records_limit=5
    )
    print(f"Snapshot: {snapshot_id}")
    
    data = await client.datasets.crunchbase_companies.download(snapshot_id)

print(f"\nDownloaded {len(data)} companies:")
for company in data:
    print(f"  - {company.get('name', 'N/A')}")
    print(f"    IPO Status: {company.get('ipo_status', 'N/A')}")
    print(f"    Stock Symbol: {company.get('stock_symbol', 'N/A')}")

---
## Test 8: Export Results

In [ ]:
import json
from pathlib import Path

if data:
    output_file = Path.cwd() / "crunchbase_dataset_results.json"
    
    with open(output_file, "w") as f:
        json.dump(data, f, indent=2, default=str)
    
    print(f"Exported to: {output_file}")
    print(f"Records: {len(data)}")
else:
    print("No data to export")

---
## Summary

### Crunchbase Companies Dataset

| Info | Value |
|------|-------|
| Dataset ID | `gd_l1vijqt9jfj7olije` |
| Records | 2.3M+ companies |
| Total Fields | 98 |
| Access | `client.datasets.crunchbase_companies` |

### Key Fields for Filtering

| Field | Type | Fill Rate | Description |
|-------|------|-----------|-------------|
| `name` | text | 100% | Company name |
| `operating_status` | text | 100% | active, closed, etc. |
| `ipo_status` | text | 99.9% | public, private, etc. |
| `country_code` | text | 93.5% | Country code |
| `cb_rank` | number | 97% | Crunchbase rank |
| `num_employees` | text | 86.3% | Employee count range |
| `num_employee_profiles` | number | 99.9% | LinkedIn profiles |
| `num_investors` | number | 8.2% | Investor count |
| `industries` | array | 94.5% | Industry categories |

### Example Filters

```python
# Active companies
{"name": "operating_status", "operator": "=", "value": "active"}

# Public companies
{"name": "ipo_status", "operator": "=", "value": "public"}

# US-based companies
{"name": "country_code", "operator": "=", "value": "USA"}

# Companies with funding
{
    "operator": "and",
    "filters": [
        {"name": "operating_status", "operator": "=", "value": "active"},
        {"name": "num_investors", "operator": ">", "value": 0}
    ]
}
```

### Helper Methods

| Method | Description |
|--------|-------------|
| `get_field_names()` | List all 98 field names |
| `get_high_fill_rate_fields(min_rate)` | Fields above fill rate threshold |
| `get_fields_by_type(type)` | Fields of specific type |